1\. **Pandas DataFrame**

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every `x` BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [1]:
# If haven't downloaded it yet, please get the data file with wget
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

--2022-11-18 15:56:43--  https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt
Risoluzione di www.dropbox.com (www.dropbox.com)... 162.125.69.18
Connessione a www.dropbox.com (www.dropbox.com)|162.125.69.18|:443... connesso.
Richiesta HTTP inviata, in attesa di risposta... 302 Found
Posizione: /s/raw/xvjzaxzz3ysphme/data_000637.txt [segue]
--2022-11-18 15:56:44--  https://www.dropbox.com/s/raw/xvjzaxzz3ysphme/data_000637.txt
Riutilizzo della connessione esistente a www.dropbox.com:443.
Richiesta HTTP inviata, in attesa di risposta... 302 Found
Posizione: https://uc12651110cf1afa7850cfa6f1c5.dl.dropboxusercontent.com/cd/0/inline/Bw9RC6pX1JNSuGpT7V3xbtpl8SMliPCzu12CahGz_1FXpS2H9VYbHqtqnXelYsI70kNUr3ZYA1Mjg6An3GRUcpyAf9x8Ms01na5Cn4bElIRkPNPUAaw6e9y1CxPuOuP2-HB8UJIQw_2a03Mofo_-QumcVR-8ROdbJDzlVvUco_xlwQ/file# [segue]
--2022-11-18 15:56:44--  https://uc12651110cf1afa7850cfa6f1c5.dl.dropboxusercontent.com/cd/0/inline/Bw9RC6pX1JNSuGpT7V3xbtpl8SMliPCzu12CahGz_1FXpS2H9VYbHqtqnXelYsI70kNUr3Z

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [2]:
import pandas as pd
import numpy as np

In [36]:
N = 1310720
data = pd.read_csv('data/data_000637.txt', nrows=N)
df = pd.DataFrame(data=data)
df.head()

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25


2\. Estimate the number of BX in a ORBIT (the value `x`).

*Hint*: check when the BX counter reaches the maximum value before being reset to 0.

In [8]:
BX = []

for i in range(N-1):
    if df.at[i, 'BX_COUNTER'] > df.at[i+1, 'BX_COUNTER'] and df.at[i, 'ORBIT_CNT'] < df.at[i+1, 'ORBIT_CNT']:
        if df.at[i+1, 'BX_COUNTER'] == 0:
            BX.append(df.at[i, 'BX_COUNTER'])
            
print("number of BX in a ORBIT: ",max(BX))  

number of BX in a ORBIT:  3563


3\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition, and convert the new column to a Time Series.

In [37]:
import datetime as dt
from datetime import timezone

itime = dt.datetime.now()
print("begin time:", itime)
itime1 = itime.replace(tzinfo=timezone.utc).timestamp()*10**9

time_ns = []

for i in range(N):
    time_ns.append(itime1+(df.at[i, 'ORBIT_CNT']*3563*25+df.at[i, 'BX_COUNTER']*25+df.at[i, 'TDC_MEAS']*25/30)*10**(-9))

time_ns = np.array(time_ns)
time_ns = pd.Series(time_ns, copy=False)
time_ns = pd.to_datetime(time_ns)
df['TIME_NS'] = time_ns

#ftime = dt.datetime.now()
#print("End time:", ftime)
#print("Elapsed time:", ftime - itime)

df.head()

begin time: 2022-11-25 11:42:44.839220


,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,TIME_NS
0,1,0,123,3869200167,2374,26,2022-11-25 11:42:44.839564544
1,1,0,124,3869200167,2374,27,2022-11-25 11:42:44.839564544
2,1,0,63,3869200167,2553,28,2022-11-25 11:42:44.839564544
3,1,0,64,3869200167,2558,19,2022-11-25 11:42:44.839564544
4,1,0,64,3869200167,2760,25,2022-11-25 11:42:44.839564544


4\. Find out the duration of the data taking in hours, minutes and seconds, by using the features of the Time Series. Perform this check reading the whole dataset.

In [38]:
tstamp = pd.Timestamp(itime)

how_much_time = []

for i in range(N):
    how_much_time.append(time_ns[i]-tstamp)
    
how_much_time = np.array(how_much_time)
how_much_time = pd.Series(how_much_time, copy=False) 

how_much_time.head()

0   0 days 00:00:00.000344544
1   0 days 00:00:00.000344544
2   0 days 00:00:00.000344544
3   0 days 00:00:00.000344544
4   0 days 00:00:00.000344544
dtype: timedelta64[ns]

5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [26]:
df1 = df.groupby("TDC_CHANNEL").count()

df2 = df1[df1.HEAD == max(df1.HEAD)]
df3 = df1[df1.HEAD < max(df1.HEAD)]
df4 = df3[df3.HEAD == max(df3.HEAD)]
df5 = df3[df3.HEAD < max(df3.HEAD)]
df6 = df5[df5.HEAD == max(df5.HEAD)]
print("the noisy channels are, in order:\n",df2.index[0],"\n",df4.index[0],"\n",df6.index[0])
#the noisy channels are 139,64,63

the noisy channels are, in order:
 139 
 64 
 63


6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit).

In [28]:
'''
non_empty_ORBITS = 0

for i in range(N-1):
    if df.at[i, 'BX_COUNTER'] > df.at[i+1, 'BX_COUNTER'] and df.at[i, 'ORBIT_CNT'] < df.at[i+1, 'ORBIT_CNT']:
         non_empty_ORBITS += 1

print("number of non-empty orbits: ",non_empty_ORBITS)
'''

df7 = df.groupby("ORBIT_CNT", as_index=False).count()
print("number of non-empty orbits: ",len(df7.index))

number of non-empty orbits:  11001


7\. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [29]:
df8 = df.loc[df["TDC_CHANNEL"] == 139]
df9 = df8.groupby("ORBIT_CNT",as_index=False).count()

print("number of unique orbits with at least one measurement from TDC_CHANNEL=139: ",len(df9.index)) 

number of unique orbits with at least one measurement from TDC_CHANNEL=139:  10976


8\. Create two Series (one for each FPGA) that have the TDC channel as index, and the number of counts for the corresponding TDC channel as values.

In [35]:
df10 = df.loc[df["FPGA"] == 0]   
df11 = df.loc[df["FPGA"] == 1]

df12 = df10.groupby("TDC_CHANNEL", as_index=False).count()
df13 = df11.groupby("TDC_CHANNEL", as_index=False).count()

dict2 = {}
for i in range(len(df12.index)):
    dict2[df12.at[i, "TDC_CHANNEL"]] = df12.at[i, "FPGA"]

dict3 = {}
for i in range(len(df13.index)):
    dict3[df13.at[i, "TDC_CHANNEL"]] = df13.at[i, "FPGA"]   

Series2 = pd.Series(dict2)
Series3 = pd.Series(dict3)
print(Series2.head(),"\n")
print(Series3.head())

1    1215
2    1602
3    1493
4    2352
5    1432
dtype: int64 

1    28438
2    32669
3    21970
4    26403
5    15003
dtype: int64


9\. **Optional:** Create two histograms (one for each FPGA) that show the number of counts for each TDC channel.